In [1]:
import pandas as pd
import os
import shutil
import numpy as np
import glob
import gzip

from QC.utils import shell_do
# from QC.imputation import impute_data_prep
# import QC.config as config

Plink is found
Plink2 is found


In [2]:
# some setup
# get precompiled binaries
#!cd /data/vitaled2/bin; wget https://software.broadinstitute.org/software/gtc2vcf/gtc2vcf_1.13-20211015.zip; unzip gtc2vcf_1.13-20211015.zip
# set path to gtc2vcf plugin
# !export BCFTOOLS_PLUGINS="/data/vitaled2/bin"


# download ref to create gtc
# !wget https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/001/405/GCF_000001405.25_GRCh37.p13/GCF_000001405.25_GRCh37.p13_genomic.fna.gz -P ../ref
# !gunzip ../ref/GCF_000001405.25_GRCh37.p13_genomic.fna.gz
# !cp ../ref/GCF_000001405.25_GRCh37.p13_genomic.fna ../ref/GCF_000001405.25_GRCh37.p13_genomic.fa

# get hg38 reference
# !wget https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/001/405/GCF_000001405.26_GRCh38/GCF_000001405.26_GRCh38_genomic.fna.gz -P ../ref
# !gunzip ../ref/GCF_000001405.26_GRCh38_genomic.fna.gz
# !cp ../ref/GCF_000001405.26_GRCh38_genomic.fna ../ref/GCF_000001405.26_GRCh38_genomic.fa

In [3]:
idat_path = '/data/CARD/PD/GP2/raw_genotypes/GP2_idats'
# gtc_path = '/data/CARD/PD/GP2/raw_genotypes/GP2_gtcs'
cnv_path = '/data/CARD/PD/GP2/cnvs'
# vcf_path = '/data/CARD/PD/GP2/raw_genotypes/GP2_vcfs'

ilmn_files_path = '/data/CARD/PD/GP2/ilmn_files'
# A1 is hg19, A2 is hg38. csv needed for indel calls
# bpm = f'{ilmn_files_path}/NeuroBooster_20042459_A1.bpm'
bpm_csv = f'{ilmn_files_path}/NeuroBooster_20042459_A2.csv' 
bpm = f'{ilmn_files_path}/NeuroBooster_20042459_A2.bpm'
egt = f'{ilmn_files_path}/NBSCluster_file_n1393_011921.egt'
iaap = f'{ilmn_files_path}/iaap-cli/iaap-cli'

key_dir = '/data/CARD/PD/GP2/key_files'
clin_dir = '/data/CARD/PD/GP2/clinical'

key_file = f'/data/CARD/PD/GP2/clinical/key_merge/GP2_master_key_full.txt'
clin_file = f'{clin_dir}/GP2_clinical.csv'

swarm_scripts_dir = f'/data/CARD/PD/GP2/swarm_scripts'


GTCtoVCF = '../GTCtoVCF/gtc_to_vcf.py'
ref_fasta = '/data/vitaled2/ref/GRCh38/GTCtoVCF_GRCh38_ref.fa'


key = pd.read_csv(f'{key_file}', sep='\t')
key.loc[:,'filename'] = key.loc[:,'SentrixBarcode_A'].astype(str) + '_' + key.loc[:,'SentrixPosition_A'].astype(str)

# define chromosomes for later use
chroms = [str(x) for x in range(1,23)]

grch38_fasta = '/data/vitaled2/ref/GRCh38/GCA_000001405.15_GRCh38_no_alt_analysis_set.fna'

genes = pd.read_csv('/data/CARD/PD/GP2/ref_panel/glist-hg38', sep='\s+', header=None, names=['chr','start','end','symbol'], dtype={'chr':str,'start':int,'end':int})
genes.columns = ['CHR','START','STOP','NAME']
genes_out = genes.loc[genes.CHR.isin(chroms)]
genes_out[['NAME','CHR','START','STOP']].to_csv('/data/CARD/PD/GP2/ref_panel/glist_hg38_intervals.csv', index=False)
# gene_list originally from plink glist
gene_list = '/data/CARD/PD/GP2/ref_panel/glist_hg38_intervals.csv'

# Call CNVs

In [4]:
# get snp metrics
with open(f'{swarm_scripts_dir}/snp_metrics.swarm', 'w') as f:
    for code in key.SentrixBarcode_A.unique():
        idat_path_ = f'{idat_path}/{code}'
        metrics_out = f'{idat_path}'
        cmd = f'\
python /data/vitaled2/GenoTools/run_snp_metrics_pipeline.py \
--idat_path {idat_path_} \
--bpm {bpm} \
--bpm_csv {bpm_csv} \
--egt {egt} \
--ref_fasta {ref_fasta} \
--out_path {metrics_out} \
--iaap {iaap}'
        f.write(f'{cmd}\n')
f.close()

In [32]:
!swarm -f {swarm_scripts_dir}/snp_metrics.swarm -g 16 -t 16 --time=02:00:00 --logdir {swarm_scripts_dir}/logs --gres=lscratch:20 --partition=norm

39438317


In [74]:
# with open(f'{swarm_scripts_dir}/snp_metrics2.swarm', 'w') as f:
#     for code in metric_fails_list:
#         idat_path_ = f'{idat_path}/{code}'
#         metrics_out = f'{idat_path}'
#         cmd = f'\
# python /data/vitaled2/GenoTools/run_snp_metrics_pipeline.py \
# --idat_path {idat_path_} \
# --bpm {bpm} \
# --bpm_csv {bpm_csv} \
# --egt {egt} \
# --ref_fasta {ref_fasta} \
# --out_path {metrics_out} \
# --iaap {iaap}'
#         f.write(f'{cmd}\n')
# f.close()

In [75]:
# !swarm -f {swarm_scripts_dir}/snp_metrics2.swarm -g 16 -t 16 --time=02:00:00 --logdir {swarm_scripts_dir}/logs --gres=lscratch:20 --partition=norm

36975418


In [5]:
# get QC-passing snps and samples
# labels = ['AAC','AFR','AJ','EAS','EUR','FIN','SAS']
# suffixes = ['bed','bim','fam']

# for label in labels:
#     for suffix in suffixes:
#         !cp /data/CARD/PD/GP2/genotypes/GP2/round2/clean/GP2_round2_{label}.{suffix} {cnv_path}
    
# # now grab AMR and KZ
# for suffix in suffixes:
#     !cp /data/CARD/PD/GP2/genotypes/GP2/round2/temp/GP2_round2_AMR_KZ.{suffix} {cnv_path}
#     !cp /data/CARD/PD/GP2/genotypes/GP2/round2/temp/GP2_round2_AMR.{suffix} {cnv_path}

# # make individual ancestry group directories
# labels = ['AAC','AFR','AJ','EAS','EUR','FIN','SAS','AMR','AMR_KZ']

# for label in labels:
#     !mkdir {cnv_path}/{label}


In [5]:
chroms = [str(i) for i in range(1,23)]

release2_cohorts = ['BCM','UMD','SYNAPS-KZ','MDGAP-QSBB','CORIELL']
labels = ['AAC','AFR','AJ','EAS','EUR','FIN','SAS','AMR','AMR_KZ']

release2_key = key.loc[key.study.isin(release2_cohorts)]
release2_key[['FID','GP2sampleID']].to_csv(f'{cnv_path}/release2.samples', sep='\t', header=False, index=False)
release2_key[['GP2sampleID','IID']].to_csv(f'{cnv_path}/release2_sample_id_key.csv')
release2_covars = release2_key.loc[:,['FID', 'GP2sampleID','sex_for_qc', 'age', 'age_of_onset']]

In [6]:
# snp QC

for label in labels:
    geno = f'{cnv_path}/{label}/GP2_round2_{label}'

    cmd1 = f'\
plink \
--bfile {geno} \
--keep {cnv_path}/release2.samples \
--make-bed \
--out {geno}_release2'

    cmd2 = f'\
plink \
--bfile {geno}_release2 \
--maf 0.01 \
--geno 0.01 \
--hwe 5e-6 \
--autosome \
--make-bed \
--out {geno}_release2_clean'

    cmd3 = f'\
plink \
--bfile {geno}_release2_clean \
--pca \
--out {geno}_release2_clean'

    cmds = [cmd1, cmd2, cmd3]

#     for cmd in cmds:
#         shell_do(cmd)

    pcs = pd.read_csv(f'{geno}_release2_clean.eigenvec', sep='\s+')
    pc_num = pcs.iloc[:, 2:].shape[1]
    pc_names = ['FID','GP2sampleID'] + [f'PC{i}' for i in range(1, pc_num+1)]
    pcs.columns = pc_names

    cov = pcs.merge(release2_covars, on=['FID','GP2sampleID'], how='left')
    cov.age.fillna(cov.age.mean(), inplace=True)
    cov.age_of_onset.fillna(cov.age_of_onset.mean(), inplace=True)
    cov.sex_for_qc.fillna(cov.sex_for_qc.median(), inplace=True)
    cov.rename(columns={'GP2sampleID':'sampleid','sex_for_qc':'sex'}, inplace=True)
    cov.to_csv(f'{geno}_release2_clean.cov', sep='\t', header=True, index=False)

    samples = cov.merge(release2_key[['GP2sampleID','IID']], left_on='sampleid', right_on='GP2sampleID', how='left')
    samples['IID'].to_csv(f'{geno}_release2_barcode.samples', header=False, index=False)

    bim = pd.read_csv(f'{geno}_release2_clean.bim', sep='\s+', header=None, names=['chr','id','pos','bp','a1','a2'], usecols=['id'])
    bim.to_csv(f'{geno}_release2_clean.snps', sep='\t', header=False, index=False)
        

        
# skip ld pruning until we have better quality rare calls
# --exclude exclusion_regions_hg38.txt \


#         cmd3 = f'\
# plink \
# --bfile {geno}_release2_maf_geno_hwe \
# --indep-pairwise 1000 10 0.02 \
# --autosome \
# --out {geno}_release2_prune'
    
#         cmd4 = f'\
# plink \
# --bfile {geno}_release2_maf_geno_hwe \
# --extract {geno}_release2_prune.prune.in \
# --make-bed \
# --out {geno}_release2_clean'

In [7]:
# now, select snps of interest and combine per-chrom per-sample into per-sample files
labels = ['AAC','AFR','AJ','EAS','EUR','FIN','SAS','AMR','AMR_KZ']
with open(f'{swarm_scripts_dir}/cnv_prep.swarm', 'w') as f:
    for label in labels:
        geno = f'{cnv_path}/{label}/GP2_round2_{label}'
        bim_path = f'{geno}_release2_clean.bim'
        samples_path = f'{geno}_release2_barcode.samples'


        samples = pd.read_csv(f'{geno}_release2_barcode.samples', header=None, names=['IID'])
#         bim = pd.read_csv(f'{geno}_release2_clean.bim', sep='\s+', header=None, names=['chr','id','pos','bp','a1','a2'], usecols=['id'])

        for sample in samples.IID.unique():
            
            code = sample.split('_')[0]
            total_mfile = f'{idat_path}/{code}/snp_metrics_{sample}_total.csv'
            cmd = f'\
python /data/vitaled2/GenoTools/run_cnv_prep_pipeline.py \
--idat_path {idat_path} \
--sample {sample} \
--bim {geno}_release2_clean.bim \
--out_path {total_mfile}'
            f.write(f'{cmd}\n')
f.close()
        
        
        
#             for chrom in chroms:

#                 mfile = f'{idat_path}/{code}/snp_metrics_{sample}_chr{chrom}.csv'
#                 metrics = pd.read_csv(mfile)
#                 metrics_tmp = metrics.loc[metrics.snpID.isin(snps.id)]
#                 total_sample_df = total_sample_df.append(metrics_tmp)

#             total_sample_df.to_csv(total_mfile, index=False, header=True)

#             out_file = f'{cnv_path}/{label}/CNV_{sample}.csv'
#             intervals = f'/data/CARD/PD/GP2/ref_panel/glist_hg38_intervals.csv'

            
            
            


In [91]:
!swarm -f {swarm_scripts_dir}/cnv_prep.swarm -g 16 -t 4 --time=1:00:00 --logdir {swarm_scripts_dir}/logs --gres=lscratch:20 --partition=norm

39522679


In [16]:
total_mfile
samples

,IID
0,206046180003_R04C01
1,206046180003_R05C01
2,206046180003_R06C01
3,206046180003_R07C01
4,206046180003_R08C01
...,...
138,206046190194_R04C01
139,206046190194_R05C01
140,206046190194_R06C01
141,206046190194_R07C01


In [6]:
with open(f'{swarm_scripts_dir}/cnvs.swarm', 'w') as f:
    labels = ['AAC','AFR','AJ','EAS','EUR','FIN','SAS','AMR','AMR_KZ']
    for label in labels:
        geno = f'{cnv_path}/{label}/GP2_round2_{label}'
        bim_path = f'{geno}_release2_clean.bim'
        samples_path = f'{geno}_release2_barcode.samples'

        
        samples = pd.read_csv(f'{geno}_release2_barcode.samples', header=None, names=['IID'])
#         bim = pd.read_csv(f'{geno}_release2_clean.bim', sep='\s+', header=None, names=['chr','id','pos','bp','a1','a2'], usecols=['id'])
#         idat_path = idat_path
        
        for sample in samples.IID.unique():
            
            code = sample.split('_')[0]
            total_mfile = f'{idat_path}/{code}/snp_metrics_{sample}_total.csv'
            
            out_file = f'{cnv_path}/{label}/CNV_{sample}.csv'
            intervals = f'/data/CARD/PD/GP2/ref_panel/glist_hg38_intervals.csv'
            cmd = f'\
python /data/vitaled2/GenoTools/run_cnv_pipeline.py \
--metrics {total_mfile} \
--out_path {out_file} \
--intervals {intervals} \
--min_variants 10 \
--kb_window 250'
            
            f.write(f'{cmd}\n')
f.close()


In [25]:
!swarm -f {swarm_scripts_dir}/cnvs.swarm --time=40:00 --logdir {swarm_scripts_dir}/logs --gres=lscratch:20 --partition=norm

39567615


## Create Dosage Matrices

In [7]:
with open(f'{swarm_scripts_dir}/cnv_dosages.swarm', 'w') as f:
    for label in labels:
        geno = f'{cnv_path}/{label}/GP2_round2_{label}'
        dosage_cmd = f'\
python /data/vitaled2/GenoTools/run_cnv_dosage_pipeline.py \
--metrics_in {cnv_path}/{label} \
--samples {geno}_release2_barcode.samples \
--out_path {geno}'
        print(dosage_cmd)
        f.write(f'{dosage_cmd}\n')
f.close()

python /data/vitaled2/GenoTools/run_cnv_dosage_pipeline.py --metrics_in /data/CARD/PD/GP2/cnvs/AAC --samples /data/CARD/PD/GP2/cnvs/AAC/GP2_round2_AAC_release2_barcode.samples --out_path /data/CARD/PD/GP2/cnvs/AAC/GP2_round2_AAC
python /data/vitaled2/GenoTools/run_cnv_dosage_pipeline.py --metrics_in /data/CARD/PD/GP2/cnvs/AFR --samples /data/CARD/PD/GP2/cnvs/AFR/GP2_round2_AFR_release2_barcode.samples --out_path /data/CARD/PD/GP2/cnvs/AFR/GP2_round2_AFR
python /data/vitaled2/GenoTools/run_cnv_dosage_pipeline.py --metrics_in /data/CARD/PD/GP2/cnvs/AJ --samples /data/CARD/PD/GP2/cnvs/AJ/GP2_round2_AJ_release2_barcode.samples --out_path /data/CARD/PD/GP2/cnvs/AJ/GP2_round2_AJ
python /data/vitaled2/GenoTools/run_cnv_dosage_pipeline.py --metrics_in /data/CARD/PD/GP2/cnvs/EAS --samples /data/CARD/PD/GP2/cnvs/EAS/GP2_round2_EAS_release2_barcode.samples --out_path /data/CARD/PD/GP2/cnvs/EAS/GP2_round2_EAS
python /data/vitaled2/GenoTools/run_cnv_dosage_pipeline.py --metrics_in /data/CARD/PD/GP2

In [18]:
!swarm -f {swarm_scripts_dir}/cnv_dosages.swarm -g 10 --time=8:00:00 --logdir {swarm_scripts_dir}/logs --gres=lscratch:20 --partition=norm

39683677


In [50]:
for label in labels:
    for dose in dosage_files:

        dosagefile = f'{cnv_path}/{label}/GP2_round2_{label}_{dose}.csv'
        !module load google-cloud-sdk/359.0.0; gsutil cp {dosagefile} gs://gp2tier2/release2_26042022/cnvs

[-] Unloading python 3.8  ... 
[+] Loading python 3.7  ... 
[+] Loading google-cloud-sdk  359.0.0 

The following have been reloaded with a version change:
  1) python/3.8 => python/3.7

Copying file:///data/CARD/PD/GP2/cnvs/AAC/GP2_round2_AAC_BAF.csv [Content-Type=text/csv]...
\ [1 files][ 72.5 MiB/ 72.5 MiB]                                                
Operation completed over 1 objects/72.5 MiB.                                     
[-] Unloading python 3.8  ... 
[+] Loading python 3.7  ... 
[+] Loading google-cloud-sdk  359.0.0 

The following have been reloaded with a version change:
  1) python/3.8 => python/3.7

Copying file:///data/CARD/PD/GP2/cnvs/AAC/GP2_round2_AAC_L2R_DUP.csv [Content-Type=text/csv]...
==> NOTE: You are uploading one or more large file(s), which would run          
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, n

In [64]:
# copy list of sampleids per ancestry group from previous release

for label in labels:
    sam_path = f'/data/CARD/PD/GP2/genotypes/GP2/round1/clean/GP2_round1_{label}_release1.psam'
    sam_out_path = f'/data/CARD/PD/GP2/genotypes/GP2/round2/clean/GP2_{label}_release1.samples'
    if os.path.isfile(sam_path):
        sam = pd.read_csv(sam_path, sep='\t')
        sam['#IID'].to_csv(sam_out_path, index=False)
        print(label)
        print()
        !module load google-cloud-sdk/359.0.0; gsutil cp {sam_out_path} gs://gp2tier2/release2_26042022/meta_data

AAC

[-] Unloading python 3.8  ... 
[+] Loading python 3.7  ... 
[+] Loading google-cloud-sdk  359.0.0 

The following have been reloaded with a version change:
  1) python/3.8 => python/3.7

Copying file:///data/CARD/PD/GP2/genotypes/GP2/round2/clean/GP2_AAC_release1.samples [Content-Type=application/octet-stream]...
/ [1 files][  2.7 KiB/  2.7 KiB]                                                
Operation completed over 1 objects/2.7 KiB.                                      
AJ

[-] Unloading python 3.8  ... 
[+] Loading python 3.7  ... 
[+] Loading google-cloud-sdk  359.0.0 

The following have been reloaded with a version change:
  1) python/3.8 => python/3.7

Copying file:///data/CARD/PD/GP2/genotypes/GP2/round2/clean/GP2_AJ_release1.samples [Content-Type=application/octet-stream]...
/ [1 files][  8.9 KiB/  8.9 KiB]                                                
Operation completed over 1 objects/8.9 KiB.                                      
EAS

[-] Unloading python 3.8  ... 


## update to gp2 ids and grab phenotypes

In [29]:
labels = ['AAC','AFR','AJ','EAS','EUR','FIN','SAS','AMR','AMR_KZ']
# labels = ['EUR']
dosage_files = ['BAF','L2R_DUP','L2R_DEL']
for label in labels:
    for dose in dosage_files:

        dosagefile = f'{cnv_path}/{label}/GP2_round2_{label}_{dose}.csv'
        dosagefile_out = f'{cnv_path}/{label}/GP2_round2_{label}_{dose}_gp2ids.csv'
        pheno_out = f'{cnv_path}/{label}/GP2_round2_{label}_{dose}.pheno'

        dosage = pd.read_csv(dosagefile)
        dosage_merge = dosage.merge(release2_key[['GP2sampleID','IID']], left_on='sampleid', right_on='IID')
        dosage_out = dosage_merge.drop(columns=['sampleid','IID']).set_index('GP2sampleID').reset_index().rename(columns={'GP2sampleID':'sampleid'})
        dosage_out.to_csv(dosagefile_out, header=True, index=False)


        dosage_pheno = dosage_out.merge(key.loc[:,['GP2sampleID','pheno']], left_on='sampleid', right_on='GP2sampleID', how='left')
        dosage_pheno2 = dosage_pheno.drop(columns=["GP2sampleID"])
        dosage_pheno_out = dosage_pheno2.loc[dosage_pheno2.pheno != -9]
        dosage_pheno_out.loc[:,'pheno'] = np.where(dosage_pheno_out.pheno == 1, 0, 1)
        dosage_pheno_out[['sampleid','pheno']].to_csv(pheno_out, sep='\t', header=True, index=False)




/tmp/ipykernel_78775/228289019.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dosage_pheno_out.loc[:,'pheno'] = np.where(dosage_pheno_out.pheno == 1, 0, 1)
/tmp/ipykernel_78775/228289019.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dosage_pheno_out.loc[:,'pheno'] = np.where(dosage_pheno_out.pheno == 1, 0, 1)
/tmp/ipykernel_78775/228289019.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

S

In [60]:
!ls /data/CARD/PD/GP2/cnvs/GP2_round2*.cov

ls: cannot access /data/CARD/PD/GP2/cnvs/GP2_round2*.cov: No such file or directory


## Generate CNV-WAS summary stats

In [18]:
dosage_files = ['BAF','L2R_DUP','L2R_DEL']
labels = ['AAC','AFR','AJ','EAS','EUR','FIN','SAS','AMR','AMR_KZ']
with open(f'{swarm_scripts_dir}/cnvwas.swarm', 'w') as f:
    for label in labels:
        for dose in dosage_files:

            dosagefile = f'{cnv_path}/{label}/GP2_round2_{label}_{dose}_gp2ids.csv'
            pheno = f'{cnv_path}/{label}/GP2_round2_{label}_{dose}.pheno'
            covar = f'/data/CARD/PD/GP2/cnvs/{label}/GP2_round2_{label}_release2_clean.cov'
            out_path = f'{cnv_path}/{label}/GP2_round2_{label}_{dose}_gp2ids_cnvwas.tab'

            cmd = f'\
python /data/vitaled2/GenoTools/run_cnvwas_pipeline.py \
--cnv_dosage_file {dosagefile} \
--out_path {out_path} \
--pheno {pheno} \
--covar {covar}'
            
            f.write(f'{cmd}\n')
f.close()



In [44]:
!swarm -f {swarm_scripts_dir}/cnvwas.swarm -g 16 -t 16 --time=2:00:00 --logdir {swarm_scripts_dir}/logs --gres=lscratch:20 --partition=norm

39713742


In [43]:
!ls /data/CARD/PD/GP2/cnvs/EUR/GP2_round2_EUR


_L2R_DEL_gp2ids_cnvwas.tab

ls: cannot access /data/CARD/PD/GP2/cnvs/EUR/GP2_round2_EUR_L2R_DEL_gp2ids_cnvwas.tab: No such file or directory


In [16]:
# !cat {swarm_scripts_dir}/logs/swarm_39707763_0.e
!ls {cnv_path}

AAC  AMR_KZ  GP2_round2_AMR_KZ_L2R_DEL_gp2ids_cnvwas.tab
AFR  EAS     release2_sample_id_key.csv
AJ   EUR     release2.samples
AMR  FIN     SAS


In [48]:
for label in labels:
    for dose in dosage_files:
        tab = f'{cnv_path}/{label}/GP2_round2_{label}_{dose}_gp2ids_cnvwas.tab'
        test_sumstat = pd.read_csv(tab, sep='\t')
        n_hits = test_sumstat[test_sumstat.P_VAL<=0.05/25000].shape[0]
        print(f'{label}, {dose}:')
        print(f'{n_hits} hits, {calculate_inflation(test_sumstat.P_VAL)} lambda')
        print()
        print()
        

AAC, BAF:

RUNNING: lambda_calculation

91 hits, {'pass': True, 'step': 'lambda_calculation', 'metrics': {'inflation': 6.948386823717885}} lambda


AAC, L2R_DUP:

RUNNING: lambda_calculation

1239 hits, {'pass': True, 'step': 'lambda_calculation', 'metrics': {'inflation': 2.076179898909073}} lambda


AAC, L2R_DEL:

RUNNING: lambda_calculation

87 hits, {'pass': True, 'step': 'lambda_calculation', 'metrics': {'inflation': 6.649193814035905}} lambda


AFR, BAF:

RUNNING: lambda_calculation

0 hits, {'pass': True, 'step': 'lambda_calculation', 'metrics': {'inflation': 2.939665992937379}} lambda


AFR, L2R_DUP:

RUNNING: lambda_calculation

0 hits, {'pass': True, 'step': 'lambda_calculation', 'metrics': {'inflation': 0.7709255766979001}} lambda


AFR, L2R_DEL:

RUNNING: lambda_calculation

0 hits, {'pass': True, 'step': 'lambda_calculation', 'metrics': {'inflation': 1.785299993604024}} lambda


AJ, BAF:

RUNNING: lambda_calculation

0 hits, {'pass': True, 'step': 'lambda_calculation', 'met

FileNotFoundError: [Errno 2] No such file or directory: '/data/CARD/PD/GP2/cnvs/FIN/GP2_round2_FIN_BAF_gp2ids_cnvwas.tab'

In [46]:
from GWAS.gwas import calculate_inflation
calculate_inflation(test_sumstat.P_VAL)


RUNNING: lambda_calculation



{'pass': True,
 'step': 'lambda_calculation',
 'metrics': {'inflation': 1.1128031631306237}}

In [40]:
!ls /data/CARD/PD/GP2/cnvs/

AAC  AMR_KZ  GP2_round2_AMR_KZ_L2R_DEL_gp2ids_cnvwas.tab
AFR  EAS     release2_sample_id_key.csv
AJ   EUR     release2.samples
AMR  FIN     SAS


In [53]:
cnv_dosage_file = f'{cnv_path}/GP2_round2_AJ_chr1_BAF_gp2ids.csv'
pheno = f'{cnv_path}/GP2_round2_AJ_chr1_BAF.pheno'
covar = f'/data/CARD/PD/GP2/cnvs/GP2_round2_AJ_release2.cov'
out_path = f'{cnv_path}/GP2_round2_AJ_chr1_BAF_gp2ids_cnvwas.tab'

In [88]:
from sklearn.preprocessing import MinMaxScaler
import statsmodels.api as sm

scaler = MinMaxScaler()
dosage_df = pd.read_csv(cnv_dosage_file, sep='\t')
pheno_df = pd.read_csv(pheno, sep='\t')
covar_df = pd.read_csv(covar, sep='\t')


if covar_df.age_of_onset.isna().all():
    covar_df.drop(columns=['age_of_onset'], inplace=True)
else:
    covar_df.loc[:,'age_of_onset'] = scaler.fit_transform(covar_df[['age_of_onset']])

if covar_df.age.isna().all():
    covar_df.drop(columns=['age'], inplace=True)
else:
    covar_df.loc[:,'age'] = scaler.fit_transform(covar_df[['age']])

if covar_df.sex_for_qc.isna().all():
    covar_df.drop(columns=['sex_for_qc'], inplace=True)

covar_df.drop(columns=['FID'], inplace=True)
covar_df.rename(columns={'GP2sampleID':'sampleid'}, inplace=True)

data_df = dosage_df.merge(covar_df, on='sampleid', how='left').merge(pheno_df, on='sampleid', how='left').set_index('sampleid')
data_df = data_df.loc[~data_df.pheno.isna()].copy()

rm_pred = [f'PC{i}' for i in range(1,21)] + ['sex_for_qc','age_of_onset','age','pheno']

pred_list = [x for x in data_df.columns if x not in rm_pred]
covars_list = [x for x in data_df.columns if x not in pred_list + [f'PC{i}' for i in range(11,21)] + ['pheno']]

results = []
fails = []

for pred in range(len(pred_list)):
    pred_name = pred_list[pred]
    formula = "pheno ~ " + pred_name + " + " + ' + '.join(covars_list)

    fitted = sm.formula.glm(formula=formula, family=sm.families.Binomial(), data=data_df).fit()
    beta_coef  = fitted.params.loc[pred_name]
    beta_se  = fitted.bse.loc[pred_name]
    p_val = fitted.pvalues.loc[pred_name]

    results.append((pred_name, beta_coef, beta_se, p_val))


output = pd.DataFrame(results, columns=('PREDICTOR', 'BETA_COEF', 'BETA_SE','P_VAL'))
# output.to_csv(out_path, sep='\t', header=True, index=False)


/usr/local/Anaconda/envs/py3.8/lib/python3.8/site-packages/statsmodels/genmod/families/links.py:187: RuntimeWarning: overflow encountered in exp
  t = np.exp(-z)
/usr/local/Anaconda/envs/py3.8/lib/python3.8/site-packages/statsmodels/genmod/families/links.py:187: RuntimeWarning: overflow encountered in exp
  t = np.exp(-z)
/usr/local/Anaconda/envs/py3.8/lib/python3.8/site-packages/statsmodels/genmod/families/links.py:187: RuntimeWarning: overflow encountered in exp
  t = np.exp(-z)
/usr/local/Anaconda/envs/py3.8/lib/python3.8/site-packages/statsmodels/genmod/families/links.py:187: RuntimeWarning: overflow encountered in exp
  t = np.exp(-z)
/usr/local/Anaconda/envs/py3.8/lib/python3.8/site-packages/statsmodels/genmod/families/links.py:187: RuntimeWarning: overflow encountered in exp
  t = np.exp(-z)
/usr/local/Anaconda/envs/py3.8/lib/python3.8/site-packages/statsmodels/genmod/families/links.py:187: RuntimeWarning: overflow encountered in exp
  t = np.exp(-z)
/usr/local/Anaconda/envs/py3

In [101]:
formula

'pheno ~ ZZZ3 + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10 + sex_for_qc + age + age_of_onset'

In [71]:
fitted = sm.formula.glm(formula=formula, family=sm.families.Binomial(), data=data_df).fit()

In [81]:
data_df.pheno.value_counts()



1.0    549
0.0    199
Name: pheno, dtype: int64

In [77]:
data_df.pheno.unique()

array([1., 0.])

In [ ]:
# rerun AJ because of weird results
with open(f'{swarm_scripts_dir}/cnvwas_rerun.swarm', 'w') as f:
    for chrom in chroms:
        for dose in dosage_files:

            dosagefile = f'{cnv_path}/GP2_round2_AJ_chr{chrom}_{dose}_gp2ids.csv'
            pheno = f'{cnv_path}/GP2_round2_{label}_chr{chrom}_{dose}.pheno'
            covar = f'/data/CARD/PD/GP2/cnvs/GP2_round2_{label}_release2.cov'
            out_path = f'{cnv_path}/GP2_round2_{label}_chr{chrom}_{dose}_gp2ids_cnvwas.tab'

            cmd = f'\
python /data/vitaled2/GenoTools/run_cnvwas_pipeline.py \
--cnv_dosage_file {dosagefile} \
--out_path {out_path} \
--pheno {pheno} \
--covar {covar}'

In [52]:
!head {cnv_path}/GP2_round2_AJ_chr1_{dose}_gp2ids_cnvwas.tab

PREDICTOR	BETA_COEF	BETA_SE	P_VAL
A3GALT2	180186710452254.94	29289334.029327605	0.0
AADACL3	5303922117695628.0	37613693.839705914	0.0
AADACL4	-2172543255804456.0	31604389.111477524	0.0
ABCA4	-2607947683774295.0	46164538.55795263	0.0
ABCB10	-1697221338975949.0	31825154.885626905	0.0
ABCD3	-1045479073035545.2	43275421.36413815	0.0
ABL2	4079841517756882.0	48870073.58994558	0.0
ACADM	3647008409793140.0	46422194.46223297	0.0
ACAP3	1100951240551445.5	17150626.2179	0.0


## combine per-chromosome sumstats 

In [128]:
for label in labels:
    for dose in dosage_files:
        total_dose_cnvwas = pd.DataFrame()
        for chrom in chroms:
        
            chr_tab = f'{cnv_path}/GP2_round2_{label}_chr{chrom}_{dose}_gp2ids_cnvwas.tab'
            if os.path.isfile(chr_tab):
                cnvwas = pd.read_csv(chr_tab, sep='\t')
                total_dose_cnvwas = total_dose_cnvwas.append(cnvwas)
            
        total_dose_cnvwas.to_csv(f'{cnv_path}/GP2_round2_{label}_{dose}_gp2ids_cnvwas.tab', sep='\t', index=False)

In [ ]:
for label in labels:
    for dose in dosage_files:
        !head {cnv_path}/GP2_round2_{label}_{dose}_gp2ids_cnvwas.tab

## Meta Analysis

In [9]:
# now, select ancestry groups which contain at least 100 cases and 100 controls
cwas_labels = []
sample_n_dict = {}
for label in labels:
    fam = pd.read_csv(f'{cnv_path}/GP2_round2_{label}_release2.fam', sep='\s+', header=None, names=['FID','IID','pat','mat','sex','pheno'])
    counts = fam.pheno.value_counts()
    controls = counts[1]
    cases = counts[2]
    sample_n_dict[label] = fam.pheno.value_counts().sum()
    if (controls >= 100) & (cases >= 100):
        cwas_labels.append(label)

In [95]:
!head {cnv_path}/GP2_round2_{dose}_cnvwas_meta.tab

GENE	TEST_STAT	P_VAL
A3GALT2	-11385650.049736194	0.0
AADACL3	-3889224.7297695857	0.0
AADACL4	-7202835.722903162	0.0
ABCA4	1501406.7723340068	0.0
ABCB10	4540386.250532163	0.0
ABCD3	-5249412.19256257	0.0
ABL2	-961642.237745079	0.0
ACADM	-4386535.663012956	0.0
ACAP3	-178296.98239870404	0.0


In [100]:
# get lambdas
from scipy.stats import ncx2
def calculate_inflation(pval_array, normalize=False, ncases=None, ncontrols=None):

    # what step are we running?
    step = 'lambda_calculation'
    print()
    print(f'RUNNING: {step}')
    print()

    # need cases and controls if normalizing
    if normalize and (not ncases or not ncontrols):
        print(f'Inflation Calculation failed!')
        print(f'If normalizing, please add ncases and ncontrols')

        process_complete = False

        inflation = 0
    
    else:
        # calculate inflation
        num = ncx2.ppf(1-pval_array, 1, nc=0)
        denom = ncx2.ppf(0.5, 1, nc = 0)
        inflation = np.nanmedian(num)/denom

        # normalize when necessary
        if normalize:
            inflation1000 = 1 + (inflation -1) * (1/ncases+ 1/ncontrols)/(1/1000 + 1/1000)
            inflation = inflation1000
        
        process_complete = True

    metrics_dict = {
        'inflation': inflation
    }

    out_dict = {
        'pass': process_complete,
        'step': step,
        'metrics': metrics_dict
    }

    return out_dict


cwas_labels = ['EUR','AMR']

    
for dose in dosage_files:
    results = []
    meta_df = pd.DataFrame()
    
    outmeta = f'{cnv_path}/GP2_round2_{dose}_cnvwas_meta.tab'
    
    for label in cwas_labels:
        total_dose = pd.read_csv(f'{cnv_path}/GP2_round2_{label}_{dose}_gp2ids_cnvwas.tab', sep='\t')
        total_dose.loc[:,'label'] = label
        total_dose.loc[:,'n'] = sample_n_dict[label]
        print(label, dose)
        print(calculate_inflation(total_dose.P_VAL))
        print()

EUR BAF

RUNNING: lambda_calculation

{'pass': True, 'step': 'lambda_calculation', 'metrics': {'inflation': 1.5864463577905352}}

AMR BAF

RUNNING: lambda_calculation

{'pass': True, 'step': 'lambda_calculation', 'metrics': {'inflation': 6.3701019372708325}}

EUR L2R_DUP

RUNNING: lambda_calculation

{'pass': True, 'step': 'lambda_calculation', 'metrics': {'inflation': 5.745921156013799}}

AMR L2R_DUP

RUNNING: lambda_calculation

{'pass': True, 'step': 'lambda_calculation', 'metrics': {'inflation': 12.943540058291651}}

EUR L2R_DEL

RUNNING: lambda_calculation

{'pass': True, 'step': 'lambda_calculation', 'metrics': {'inflation': 1.4466022068531328}}

AMR L2R_DEL

RUNNING: lambda_calculation

{'pass': True, 'step': 'lambda_calculation', 'metrics': {'inflation': 23.4058805066012}}



In [92]:
# meta analyze 
import scipy

cwas_labels = ['EUR','AMR']

    
for dose in dosage_files:
    results = []
    meta_df = pd.DataFrame()
    
    outmeta = f'{cnv_path}/GP2_round2_{dose}_cnvwas_meta.tab'
    
    for label in cwas_labels:
        total_dose = pd.read_csv(f'{cnv_path}/GP2_round2_{label}_{dose}_gp2ids_cnvwas.tab', sep='\t')
        total_dose.loc[:,'label'] = label
        total_dose.loc[:,'n'] = sample_n_dict[label]
        meta_df = meta_df.append(total_dose)
#         print(meta_df)

    gene_list = meta_df.PREDICTOR.unique()
        
    for gene in gene_list:
        temp_data = meta_df[meta_df['PREDICTOR'] == gene]
        temp_data['z'] = temp_data['BETA_COEF']/temp_data['BETA_SE']
        temp_data['weighted_z'] = temp_data['z']*temp_data['n']
        temp_data['weight_sq'] = temp_data['n']*temp_data['n']
        summed_z = (temp_data['weighted_z'].sum())/(np.sqrt(temp_data['weight_sq'].sum()))
        test_stat = summed_z
        p_val = scipy.stats.norm.sf(abs(summed_z))*2
#         print(gene, test_stat, p_val)
        results.append([gene, test_stat, p_val])

    output = pd.DataFrame(results, columns=['GENE', 'TEST_STAT', 'P_VAL'])
#     output.to_csv(outmeta, sep='\t', index=False)

        
        

KeyboardInterrupt: 

In [40]:
meta_df

,PREDICTOR,BETA_COEF,BETA_SE,P_VAL,label,n
0,A3GALT2,1.801867e+14,2.928933e+07,0.000000,AJ,751
1,AADACL3,5.303922e+15,3.761369e+07,0.000000,AJ,751
2,AADACL4,-2.172543e+15,3.160439e+07,0.000000,AJ,751
3,ABCA4,-2.607948e+15,4.616454e+07,0.000000,AJ,751
4,ABCB10,-1.697221e+15,3.182515e+07,0.000000,AJ,751
...,...,...,...,...,...,...
24326,ZNF280B,9.628337e-01,1.114912e+00,0.387810,AMR,398
24327,ZNF70,-4.301046e+00,1.167099e+00,0.000228,AMR,398
24328,ZNF74,-4.769509e+00,1.294362e+00,0.000229,AMR,398
24329,ZNRF3,-8.228554e+00,1.955440e+00,0.000026,AMR,398


In [38]:
total_dose = pd.read_csv(f'{cnv_path}/GP2_round2_EUR_L2R_DEL_gp2ids_cnvwas.tab', sep='\t')
total_dose

,PREDICTOR,BETA_COEF,BETA_SE,P_VAL
0,A3GALT2,0.117337,0.302572,0.698165
1,AADACL3,-0.190159,0.399284,0.633897
2,AADACL4,0.039304,0.337910,0.907403
3,ABCA4,-0.848758,0.489625,0.083009
4,ABCB10,0.148269,0.328616,0.651850
...,...,...,...,...
24326,ZNF280B,-0.237103,0.201098,0.238380
24327,ZNF70,0.463760,0.256009,0.070063
24328,ZNF74,0.509812,0.277732,0.066413
24329,ZNRF3,0.390342,0.391069,0.318211


In [131]:
total_dose_cnvwas = pd.read_csv(f'{cnv_path}/GP2_round2_EUR_L2R_DEL_gp2ids_cnvwas.tab', sep='\t')
adj_thresh = 0.05/total_dose_cnvwas.shape[0]
total_dose_cnvwas[total_dose_cnvwas.P_VAL<=5e-7]

,PREDICTOR,BETA_COEF,BETA_SE,P_VAL
10106,ZNF138,1.74966,0.346772,4.521800e-07


In [133]:
total_dose_cnvwas = pd.read_csv(f'{cnv_path}/GP2_round2_AMR_KZ_L2R_DEL_gp2ids_cnvwas.tab', sep='\t')
total_dose_cnvwas[total_dose_cnvwas.P_VAL<=5e-7]

,PREDICTOR,BETA_COEF,BETA_SE,P_VAL
91,APH1A,-39.673290,7.664462,2.263709e-07
236,C1orf54,-38.220079,7.342070,1.933427e-07
252,CA14,-31.704818,6.278753,4.428747e-07
360,CIART,-39.661819,7.554564,1.520598e-07
1283,MIR101_1,-32.240891,6.360468,4.000177e-07
1338,MIR3671,-32.240891,6.360468,4.000177e-07
1472,MRPS21,-38.486301,7.206974,9.287115e-08
1803,PRPF3,-46.275790,8.737541,1.182356e-07
3534,MIR663B,-38.538861,7.576270,3.641833e-07
13775,MS4A13,-65.620291,12.885892,3.535338e-07


In [147]:
# get case/control counts


    
                      

AAC
1    344
2     90
Name: pheno, dtype: int64

AFR
1    133
2     20
Name: pheno, dtype: int64

AJ
 2    550
 1    200
-9      1
Name: pheno, dtype: int64

EAS
1    79
2    70
Name: pheno, dtype: int64

EUR
 2    4199
 1    2329
-9       7
Name: pheno, dtype: int64

FIN
2    6
1    1
Name: pheno, dtype: int64

SAS
2    48
1    40
Name: pheno, dtype: int64

AMR
2    229
1    169
Name: pheno, dtype: int64

AMR_KZ
1    101
2     43
Name: pheno, dtype: int64



In [149]:
test_pheno = pd.read_csv(f'{cnv_path}/GP2_round2_EUR_chr{chrom}_{dose}.pheno', sep='\t')
test_pheno.pheno.value_counts()

1    4199
0    2327
Name: pheno, dtype: int64

In [121]:
# total_dose_cnvwas.loc[:,'corrected'] = total_dose_cnvwas['P_VAL']/total_dose_cnvwas.shape[0]

In [119]:
total_dose_cnvwas[total_dose_cnvwas.corrected

3464.2402205761655

In [122]:
# from statsmodels.stats.multitest import fdrcorrection
total_dose_cnvwas


,PREDICTOR,BETA_COEF,BETA_SE,P_VAL,corrected
0,A3GALT2,-0.961458,3.278829,0.769345,3.161994e-05
1,AADACL3,-9.820355,3.749922,0.008824,3.626466e-07
2,AADACL4,-8.383166,3.476521,0.015893,6.531944e-07
3,ABCA4,-14.459306,6.007200,0.016084,6.610644e-07
4,ABCB10,12.507769,3.579374,0.000475,1.952758e-08
...,...,...,...,...,...
574,ZNF280B,-46.384276,12.057866,0.000120,4.918313e-09
575,ZNF70,10.486147,2.903098,0.000304,1.248536e-08
576,ZNF74,8.854631,2.774623,0.001416,5.821311e-08
577,ZNRF3,-11.746530,3.850717,0.002285,9.390371e-08


In [85]:
fdr_correction[0].sum()

0

In [ ]:




# from sklearn.preprocessing import MinMaxScaler
# import statsmodels.api as sm

# for label in labels:
#     geno = f'{cnv_path}/GP2_round2_{label}'

# scaler = MinMaxScaler()
# cnv_dosage_file = f'{cnv_path}/GP2_round2_EUR_chr12_L2R_DEL_gp2ids.csv'
# pheno = f'{cnv_path}/GP2_round2_EUR_chr12_L2R_DEL.pheno'
# covar = f'/data/CARD/PD/GP2/cnvs/GP2_round2_EUR_release2.cov'

# dosage_df = pd.read_csv(cnv_dosage_file, sep='\t')
# pheno_df = pd.read_csv(pheno, sep='\t')
# covar_df = pd.read_csv(covar, sep='\t')


# if covar_df.age_of_onset.isna().all():
#     covar_df.drop(columns=['age_of_onset'], inplace=True)
# else:
#     covar_df.loc[:,'age_of_onset'] = scaler.fit_transform(covar_df[['age_of_onset']])
    
# if covar_df.age.isna().all():
#     covar_df.drop(columns=['age'], inplace=True)
# else:
#     covar_df.loc[:,'age'] = scaler.fit_transform(covar_df[['age']])
    
# if covar_df.sex_for_qc.isna().all():
#     covar_df.drop(columns=['sex_for_qc'], inplace=True)

# covar_df.drop(columns=['FID'], inplace=True)
# covar_df.rename(columns={'GP2sampleID':'sampleid'}, inplace=True)

# data_df = dosage_df.merge(covar_df, on='sampleid', how='left').merge(pheno_df, on='sampleid', how='left').set_index('sampleid')
# # pred_list_tmp = [x for x in data_df.columns if x not in ['pheno']]
# rm_pred = [f'PC{i}' for i in range(1,21)] + ['sex_for_qc','age_of_onset','age','pheno']
# # rm_cols = [s for s in predictor_list_tmp if s in rm_pcs]
# pred_list = [x for x in data_df.columns if x not in rm_pred]
# covars_list = [x for x in data_df.columns if x not in pred_list + [f'PC{i}' for i in range(11,21)] + ['pheno']]

# results = []
# fails = []

# for pred in range(len(pred_list)):
#     pred_name = pred_list[pred]
#     formula = "pheno ~ " + pred_name + " + " + ' + '.join(covars_list)
# #     this_formula = "pheno ~ " + "data_df['" + pred_name + "']" + " + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + age_at_baseline + PCT_USABLE_BASES + MALE"
# #     formula = "pheno ~ " + "data_df['" + pred_name + "'] + " + ' + '.join(covars_list)
# #     try:
#     fitted = sm.formula.glm(formula=formula, family=sm.families.Binomial(), data=data_df).fit()
#     beta_coef  = fitted.params.loc[pred_name]
#     beta_se  = fitted.bse.loc[pred_name]
#     p_val = fitted.pvalues.loc[pred_name]
    
    
# # #     fitted = sm.formula.logit(formula=this_formula, data=data_df).fit()
# #     beta_coef  = fitted.params.loc["data_df['" + pred_name + "']"]
# #     beta_se  = fitted.bse.loc["data_df['" + pred_name + "']"]
# #     p_val = fitted.pvalues.loc["data_df['" + pred_name + "']"]
#     results.append((pred_name, beta_coef, beta_se, p_val))
# # #     except:
# # # #         print(f'{predictor_name} did not converge!!')
# # #         fails.append(pred_name)



# output = pd.DataFrame(results, columns=('PREDICTOR', 'BETA_COEF', 'BETA_SE','P_VAL')) 
# output





    

#     scaler = MinMaxScaler()

#     data_df = pd.read_csv(cnv_dosage_file)

#     data_df.loc[:,'pheno'] = pheno
#     # predictor_list = data_df.columns.to_list()
#     predictor_list = [x for x in data_df.columns if x not in ['pheno']]

#     results = []
#     fails = []

#     for predictor in range(len(predictor_list)):
#         predictor_name = predictor_list[predictor]
#         this_formula = "pheno ~ " + "data_df['" + predictor_list[predictor] + "']" #+ " + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + age_at_baseline + PCT_USABLE_BASES + MALE"
#         try:
#             fitted = sm.formula.glm(formula=this_formula, family=sm.families.Binomial(), data=data_df).fit()
#     #     fitted = sm.formula.logit(formula=this_formula, data=data_df).fit()
#             beta_coef  = fitted.params.loc["data_df['" + predictor_name + "']"]
#             beta_se  = fitted.bse.loc["data_df['" + predictor_name + "']"]
#             p_val = fitted.pvalues.loc["data_df['" + predictor_name + "']"]
#             results.append((predictor_name, beta_coef, beta_se, p_val))
#         except:
#             print(f'{predictor_name} did not converge!!')
#             fails.append(predictor_name)



#     output = pd.DataFrame(results, columns=('PREDICTOR', 'BETA_COEF', 'BETA_SE','P_VAL')) 


In [5]:
with open(f'{swarm_scripts_dir}/idat_to_gtc.swarm', 'w') as f:
    
    for code in key.SentrixBarcode_A.unique():
        os.makedirs(f'{gtc_path}/{code}', exist_ok=True)
        idat_to_gtc_cmd = f'\
{iaap} gencall \
{bpm} \
{egt} \
{gtc_path}/{code} \
-f {idat_path}/{code} \
-g \
-t 8'
        
        f.write(f'{idat_to_gtc_cmd}\n')
f.close()

In [39]:
!swarm -f {swarm_scripts_dir}/idat_to_gtc.swarm -g 15 -t 10 --time=10:00:00 --logdir {swarm_scripts_dir}/logs --gres=lscratch:20 --partition=norm --job-name idat_to_gtc

32381997


In [6]:
with open(f'{swarm_scripts_dir}/gtc2vcf.swarm','w') as f:
    for code in key.SentrixBarcode_A.unique():
        gtc2vcf_cmd = f'\
export BCFTOOLS_PLUGINS="/data/vitaled2/bin"; bcftools +gtc2vcf \
--no-version -Ob \
--bpm {bpm} \
--csv {bpm_csv} \
--egt {egt} \
--gtcs {gtc_path}/{code} \
--fasta-ref {grch38_fasta} \
--extra {vcf_path}/gp2_snps_{code}_metadata.tsv | \
bcftools norm --no-version -Oz -c w -f {grch38_fasta} > {vcf_path}/gp2_snps_{code}.vcf.gz'
    
        f.write(f'{gtc2vcf_cmd}\n')
f.close()


In [41]:
!swarm -f {swarm_scripts_dir}/gtc2vcf.swarm -g 64 -t 32 --time=10:00:00 --logdir {swarm_scripts_dir}/logs --gres=lscratch:20 --partition=norm --job-name gtc_to_vcf

32384390


In [7]:
# sort vcf
!mkdir {vcf_path}/tmp
with open(f'{swarm_scripts_dir}/bcftools_sort.swarm', 'w') as f:
    for code in key.SentrixBarcode_A.unique():
        sort_cmd = f'cd {vcf_path}; bcftools sort gp2_snps_{code}.vcf.gz -T ./tmp -Oz -o gp2_snps_sorted_{code}.vcf.gz'
        f.write(f'{sort_cmd}\n')
f.close()

mkdir: cannot create directory ‘/data/CARD/PD/GP2/raw_genotypes/GP2_vcfs/tmp’: File exists


In [43]:
!swarm -f {swarm_scripts_dir}/bcftools_sort.swarm -g 32 -t 16 --time=10:00:00 --logdir {swarm_scripts_dir}/logs --gres=lscratch:20 --partition=norm --job-name vcf_sort

32476908


In [8]:
# split indels and snps in vcf
with open(f'{swarm_scripts_dir}/vcftools_remove_indels.swarm', 'w') as f:
    for code in key.SentrixBarcode_A.unique():
        ext_snps_cmd = f'cd {vcf_path}; vcftools --gzvcf gp2_snps_sorted_{code}.vcf.gz --remove-indels --recode --recode-INFO-all --out gp2_snps_sorted_snps_only_{code}'
        f.write(f'{ext_snps_cmd}\n')
f.close()

In [45]:
!swarm -f {swarm_scripts_dir}/vcftools_remove_indels.swarm -g 16 -t 16 --time=10:00:00 --logdir {swarm_scripts_dir}/logs --gres=lscratch:20 --partition=norm --job-name vcf_remove_indels

32481160


In [9]:
# split indels and snps in vcf
with open(f'{swarm_scripts_dir}/vcftools_keep_indels.swarm', 'w') as f:
    for code in key.SentrixBarcode_A.unique():
        keep_indels_cmd = f'cd {vcf_path}; vcftools --gzvcf gp2_snps_sorted_{code}.vcf.gz --keep-only-indels --recode --recode-INFO-all --out gp2_snps_sorted_indels_only_{code}'
        f.write(f'{keep_indels_cmd}\n')
f.close()

In [47]:
!swarm -f {swarm_scripts_dir}/vcftools_keep_indels.swarm -g 16 -t 16 --time=10:00:00 --logdir {swarm_scripts_dir}/logs --gres=lscratch:20 --partition=norm --job-name vcf_keep_indels

32481168


In [62]:
# get snp info from each vcf
with open(f'{swarm_scripts_dir}/get_logr_baf.swarm', 'w') as f:
    for code in key.SentrixBarcode_A.unique():
        get_logr_baf = f'python3 process_vcf_snps.py --vcf {vcf_path}/gp2_snps_sorted_snps_only_{code}.recode.vcf --gene_ref /data/CARD/PD/GP2/ref_panel/glist-hg38 --out {vcf_path}/gp2_snp_metrics_{code}.txt'
        f.write(f'{get_logr_baf}\n')
f.close()

In [63]:
!swarm -f {swarm_scripts_dir}/get_logr_baf.swarm -g 64 -t 32 --time=10:00:00 --logdir {swarm_scripts_dir}/logs --gres=lscratch:20 --partition=norm --job-name get_snp_metrics

32974379


In [37]:
# get snp info from each vcf
with open(f'{swarm_scripts_dir}/clean_snp_metrics.swarm', 'w') as f:
    for code in key.SentrixBarcode_A.unique():
        clean_snp_metrics = f'python3 clean_snp_metrics.py --infile {vcf_path}/gp2_snp_metrics_{code}.txt --outpath {vcf_path}/gp2_snp_metrics'
        f.write(f'{clean_snp_metrics}\n')
f.close()

In [38]:
!swarm -f {swarm_scripts_dir}/clean_snp_metrics.swarm -g 16 -t 16 --time=10:00:00 --logdir {swarm_scripts_dir}/logs --gres=lscratch:20 --partition=norm --job-name clean_snp_metrics

34805915


In [36]:
!cat {swarm_scripts_dir}/logs/clean_snp_metrics_34805480_0.e

  File "clean_snp_metrics.py", line 11
    infile = args.in
                   ^
SyntaxError: invalid syntax
  File "clean_snp_metrics.py", line 11
    infile = args.in
                   ^
SyntaxError: invalid syntax


In [34]:
# # split each file by chr and iid

# infile = f'{vcf_path}/gp2_snp_metrics_204697840024.txt'
# df = pd.read_csv(infile, sep='\t')


# cols = ['CHROM','POS','ID','REF','ALT','sampleid','BAF','LRR']

# for iid in df.sampleid.unique():
#     for chrom in df.CHROM.unique():
        
#         outfile = f'{vcf_path}/gp2_snp_metrics_{iid}_chr{chrom}.txt' 
#         out_df = df.loc[(df.CHROM==chrom) & (df.sampleid==iid)]
#         out_df[['CHROM','ID','POS','BAF','LRR']].to_csv(outfile, sep='\t', header=True, index=False)
        

In [59]:
metrics = pd.read_csv(f'{vcf_path}/gp2_snp_metrics_{code}.txt', sep='\t', nrows=100, dtype={'CHROM':str,'POS':int})
metrics.loc[:,'CHROM'] = metrics.loc[:,'CHROM'].str.replace('chr','')
metrics.loc[:,'gene'] = np.nan

In [60]:
# for i, gene in enumerate(gene_list.symbol):
# #     print(gene_list.loc[i,:])
#     chrom = gene_list.loc[i,'chr']
#     start = gene_list.loc[i,'start'] - 100000
#     end = gene_list.loc[i, 'end'] + 100000
#     metrics.loc[((metrics.CHROM==chrom) & (metrics.POS>=start) & (metrics.POS<=end)),'gene'] = gene
    


## Dev for CNV module in GenoTools

In [25]:
# import os
# import subprocess
# from QC.utils import shell_do

# def idat_snp_metrics(idat_path, bpm, bpm_csv, egt, ref_fasta, out_path, iaap=iaap):
#     '''
#     current structure of idat storage is such that a directory of each SentrixBarcode_A with all idats for that barcode in it
#     for ex.
#     1112223334
#         --> 1112223334_R01C01_Red.idat
#         --> 1112223334_R01C01_Grn.idat
#         --> 1112223334_R01C02_Red.idat
#         --> 1112223334_R01C02_Grn.idat
#         etc.
        
#     '''
#     out_tmp = f'{out_path}/tmp'
#     os.makedirs(out_tmp, exist_ok=True)
#     barcode = idat_path.split('/')[-1].split('_')[0]
#     barcode_out_path = f'{out_path}/{barcode}'
#     os.makedirs(barcode_out_path, exist_ok=True)
    
    
#     idat_to_gtc_cmd = f'\
# {iaap} gencall \
# {bpm} \
# {egt} \
# {barcode_out_path} \
# -f {idat_path} \
# -g \
# -t 8'

#     # export path to plugins temporarily for biowulf. will figure this out later
#     gtc2vcf_cmd = f'\
# export BCFTOOLS_PLUGINS="/data/vitaled2/bin"; \
# bcftools +gtc2vcf \
# --no-version -Ob \
# --bpm {bpm} \
# --csv {bpm_csv} \
# --egt {egt} \
# --gtcs {barcode_out_path} \
# --fasta-ref {ref_fasta} | \
# bcftools norm --no-version -Oz -c w -f {ref_fasta} > {barcode_out_path}/{barcode}.vcf.gz'
    
#     # --extra {vcf_path}/gp2_snps_{code}_metadata.tsv | \

#     # sort vcf
#     sort_cmd = f'\
# bcftools \
# sort {barcode_out_path}/{barcode}.vcf.gz \
# -T {out_tmp}/ \
# -Oz -o {barcode_out_path}/{barcode}_sorted.vcf.gz'
# # cd {barcode_out_path} && \

#     # split indels and snps in vcf
#     ext_snps_cmd = f'\
# vcftools --gzvcf \
# {barcode_out_path}/{barcode}_sorted.vcf.gz \
# --remove-indels \
# --recode \
# --recode-INFO-all \
# --out {barcode_out_path}/{barcode}_sorted_snps'
# # cd {barcode_out_path} && \


#     # split indels and snps in vcf
# #     keep_indels_cmd = f'\
# # cd {barcode_out_path}; \
# # vcftools --gzvcf \
# # {barcode}_sorted.vcf.gz \
# # --keep-only-indels \
# # --recode \
# # --recode-INFO-all \
# # --out {code}_indels'

#     # get snp info from each vcf
#     get_logr_baf = f'\
# python3 process_vcf_snps.py \
# --vcf {barcode_out_path}/{barcode}_sorted_snps.recode.vcf \
# --outfile {barcode_out_path}/snp_metrics_{barcode}.csv'


#     # get snp info from each vcf
#     clean_snp_metrics = f'\
# python3 clean_snp_metrics.py \
# --infile {barcode_out_path}/snp_metrics_{barcode}.csv \
# --outfile {barcode_out_path}/snp_metrics'

#     cmds = [idat_to_gtc_cmd, gtc2vcf_cmd, sort_cmd, ext_snps_cmd, get_logr_baf, clean_snp_metrics]
# #     cmds = [get_logr_baf, clean_snp_metrics]
# #     cmds = [get_logr_baf]
# #     cmds = [clean_snp_metrics]
#     for cmd in cmds:
#         if cmd == gtc2vcf_cmd:
#             subprocess.call(cmd, shell=True)
#         else:
#             shell_do(cmd)
            
 
#     # output snp metrics paths
#     chroms = [str(i) for i in range(1,23)] + ['X','Y']
#     samples = [s.split('/')[-1].replace('_Red.idat','') for s in glob.glob(f'{idat_path}/*_Red.idat')]
    
#     outfiles = []
#     for sample in samples:
#         for chrom in chroms:
#             outfile = f'{barcode_out_path}/snp_metrics_{sample}_chr{chrom}.csv'
#             outfiles.append(outfile)
            
#     return outfiles


# for sample in samples:
#     for chrom in chroms:

#     call_cnvs_cmd = f'\
#     python3 cnv_gene_caller_alpha.py \
#     --infile {barcode_out_path}/snp_metrics_{sample}_chr{chrom}.csv \
#     --outfile {barcode_out_path}/{sample}_chr{chrom} \
#     --intervals {gene_list}'
            
#             print(call_cnvs_cmd)
#             shell_do(call_cnvs_cmd)


In [20]:
# infile = f'{test_path}/snp_metrics_206046180074.csv'
# outfile = f'{test_path}/snp_metrics'

In [70]:
!ls {idat_path}

206046180074_R01C01_Grn.idat  206046180074_R05C01_Grn.idat
206046180074_R01C01_Red.idat  206046180074_R05C01_Red.idat
206046180074_R02C01_Grn.idat  206046180074_R06C01_Grn.idat
206046180074_R02C01_Red.idat  206046180074_R06C01_Red.idat
206046180074_R03C01_Grn.idat  206046180074_R07C01_Grn.idat
206046180074_R03C01_Red.idat  206046180074_R07C01_Red.idat
206046180074_R04C01_Grn.idat  206046180074_R08C01_Grn.idat
206046180074_R04C01_Red.idat  206046180074_R08C01_Red.idat


In [58]:
idat_path = '/data/vitaled2/cnv_test/206046180074'
test_out = '/data/vitaled2/cnv_test'
# snp_metrics_files = idat_snp_metrics(idat_path, bpm, bpm_csv, egt, ref_fasta, test_out)

with open(f'{swarm_scripts_dir}/call_cnvs.swarm', 'w') as f:
    
    for mfile in snp_metrics_files:
        out_prefix = mfile.replace('.csv','').replace('snp_metrics_', '')
#         chrom = out_prefix.split('_')[-1]
#         gene_chrom_list = f'/data/CARD/PD/GP2/ref_panel/glist_hg38_{chrom}.csv'

        call_cnvs_cmd = f'\
python3 cnv_gene_caller_alpha.py \
--infile {mfile} \
--outfile {out_prefix} \
--intervals {gene_list}'

        f.write(f'{call_cnvs_cmd}\n')
f.close()

In [59]:
!swarm -f {swarm_scripts_dir}/call_cnvs.swarm -g 16 -t 16 --time=10:00:00 --logdir {swarm_scripts_dir}/logs --gres=lscratch:20 --partition=norm

35729936
